In [15]:
import os, sys
import random
import gensim
from io import StringIO
import datetime
import subprocess
import mysql.connector
from dotenv import load_dotenv
load_dotenv()
from collections import Counter
from mysql.connector import Error
from gensim.corpora import Dictionary
from gensim.corpora import MalletCorpus
import helper_functions
import pickle

In [2]:
c = []
connection = helper_functions.get_connection()
cursor = connection.cursor(buffered=True)
query = """
select fname,lname,f.aid,j.blobid,serialized_tokens from myfiles f join myapplicants a join my_job_market_papers j
on f.aid=a.aid and f.blobid=j.blobid
    """
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    r = [str(row[3]), str(row[0])+" "+str(row[1])]
    r = r+pickle.loads(row[4])
    c.append(r)

Connected to MySQL database... 


In [3]:
#c[0]

In [4]:
dictionary = Dictionary(c)


In [5]:
#corpus = [dictionary.doc2bow(doc) for doc in c]
corpus = []
id2doc = []
n = 0
for doc in c:
    corpus.append(dictionary.doc2bow(doc))
    id2doc.append([n,doc[0]])
    n +=1
                   

In [6]:
len(corpus)

748

In [13]:
#output_fname = get_tmpfile("corpus.mallet")
#MalletCorpus.serialize("corpus.mallet", corpus, dictionary)

In [12]:
#loaded_corpus = MalletCorpus("corpus.mallet")

In [11]:
#loaded_corpus[747]

In [34]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
lsi_corpus = gensim.models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=10)

In [37]:
words = "climate change carbon tax".lower().split()
print(tfidf[dictionary.doc2bow(words)])

[(187, 0.00875279906117721), (2930, 0.2487417459967813), (5992, 0.8022550551422578), (8224, 0.5426211927355584)]


In [24]:

#Serialize and Store the corpus locally for easy retrival whenver required.
#tdif_corpus = gensim.corpora.MmCorpus.serialize('tfidf_mm', tfidf[corpus])
#lsi_corpus = gensim.corpora.MmCorpus.serialize('lsi_mm',[tfidf[corpus]])

#Load the indexed corpus
#movie_tfidf_corpus = gensim.corpora.MmCorpus('movie_tfidf_model_mm')
#movie_lsi_corpus = gensim.corpora.MmCorpus('movie_lsi_model_mm')


gensim.models.lsimodel.LsiModel

In [38]:

#Load the MatrixSimilarity
from gensim.similarities import MatrixSimilarity
jmp_index = MatrixSimilarity(lsi_corpus[corpus], num_features = lsi_corpus.num_terms)

In [42]:
print(words_lsi)

[(0, 0.16674353549000043), (1, 0.021061477110922336), (2, -0.02232567454820548), (3, -0.16253345389135543), (4, 0.07306664716810304), (5, -0.054699094707779033), (6, 0.08521069278410319), (7, 0.25518264627699466), (8, 0.03922793959133026), (9, -0.36317400668761235)]


In [63]:
#sims = sorted(enumerate(sims), key=lambda item: -item[1])
words = "incentives mechanism design computation".lower().split()
words_bow = dictionary.doc2bow(words)
words_lsi = lsi_corpus[words_bow]
sims = jmp_index[words_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims[0]

(554, 0.9745666)

In [62]:
n = 0
for doc_position, doc_score in sims:
    v = c[doc_position]
    print(doc_score, v[0])
    n += 1
    if n == 5:
        break


0.9745666 1606404
0.97386414 1645134
0.97163486 1568277
0.97148454 1623869
0.9673481 1621951
